In [10]:
import pandas as pd

In [13]:
interactions = pd.read_csv("./data/interactions.csv", names=['datetime', 'viewer_id', 'viewer_name', 'profile_id','profile_name', 'status', 'score'])

In [14]:
interactions

,datetime,viewer_id,viewer_name,profile_id,profile_name,status,score
0,2025-11-07 09:45:11,Sai-f8476bcc,Sai,6f8db1aa,Kiran,pass,0.993
1,2025-11-07 09:45:12,Sai-f8476bcc,Sai,a585683b,Ethan,superlike,0.980
2,2025-11-07 09:49:28,Elena-fde9c9f3,Elena,050a9be7,Mia,pass,0.993
3,2025-11-07 09:49:30,Elena-fde9c9f3,Elena,229998fb,Aya,superlike,0.993
4,2025-11-07 09:49:32,Elena-fde9c9f3,Elena,836066e3,Emma,like,0.993
5,2025-11-07 09:49:34,Elena-fde9c9f3,Elena,71c74a49,Rohan,pass,0.993
6,2025-11-07 09:49:35,Elena-fde9c9f3,Elena,5ccd51be,Aarohi,superlike,0.993
7,2025-11-07 09:49:37,Elena-fde9c9f3,Elena,d6728d39,Jordan,like,0.993
8,2025-11-07 09:51:19,Kabir-7faa872c,Kabir,d6d104b4,Andre,pass,0.993
9,2025-11-07 09:51:20,Kabir-7faa872c,Kabir,55628e45,Zainab,superlike,0.993


In [16]:
interactions['viewer_id'] = interactions['viewer_id'].apply(lambda w: w.split("-")[-1])

In [17]:
interactions

,datetime,viewer_id,viewer_name,profile_id,profile_name,status,score
0,2025-11-07 09:45:11,f8476bcc,Sai,6f8db1aa,Kiran,pass,0.993
1,2025-11-07 09:45:12,f8476bcc,Sai,a585683b,Ethan,superlike,0.980
2,2025-11-07 09:49:28,fde9c9f3,Elena,050a9be7,Mia,pass,0.993
3,2025-11-07 09:49:30,fde9c9f3,Elena,229998fb,Aya,superlike,0.993
4,2025-11-07 09:49:32,fde9c9f3,Elena,836066e3,Emma,like,0.993
5,2025-11-07 09:49:34,fde9c9f3,Elena,71c74a49,Rohan,pass,0.993
6,2025-11-07 09:49:35,fde9c9f3,Elena,5ccd51be,Aarohi,superlike,0.993
7,2025-11-07 09:49:37,fde9c9f3,Elena,d6728d39,Jordan,like,0.993
8,2025-11-07 09:51:19,7faa872c,Kabir,d6d104b4,Andre,pass,0.993
9,2025-11-07 09:51:20,7faa872c,Kabir,55628e45,Zainab,superlike,0.993


In [67]:
update = interactions.groupby('viewer_id')['profile_id'].apply(list).reset_index()

In [38]:
import re
def str_to_array(s):
    if s is None:
        return None
    s = s.strip()
    if s.startswith('[') and s.endswith(']'):
        s = s[1:-1]
    s = re.sub(r'[\r\n]+',' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return np.fromstring(s, sep=' ').astype(float)

p = pd.read_csv("./data/profile_embedding.csv")
p['embedding'] = p['embedding'].apply(str_to_array)

In [54]:
import numpy as np
from numpy.linalg import norm

def avg(l):

    es = p[p['id'].isin(l)]['embedding'].values
    avg = np.mean(es, axis=0)
    n = avg / norm(avg)
    return n

In [73]:
update['updated_embedding'] = update['profile_id'].apply(avg)

In [75]:
update = update.rename(columns={'viewer_id': 'id'})

In [77]:
update = update.rename(columns={'updated_embedding': 'embedding'})

In [79]:
update = update[['id', 'embedding']]

In [80]:
update

,id,embedding
0,7faa872c,"[-0.021411143124228673, 0.022420545986609954, ..."
1,f8476bcc,"[-0.0348687370507647, -0.003099817113787059, 0..."
2,fde9c9f3,"[-0.0011952618732604542, 0.003903285141021589,..."
3,ssse1024,"[-0.033956613448339376, 0.034623023154416975, ..."


In [84]:
update_dict = dict(zip(update['id'], update['embedding']))

In [ ]:
# profiles_updated['embedding'] = profiles_updated.apply(
#     lambda row: update_dict.get(row['id'], row['embedding']),
#     axis=1
# )

In [87]:
p

,id,embedding
0,ssse1024,"[-0.00566114299, -0.00296856952, 0.0457694009,..."
1,d8334dc0,"[-0.00219186791, -0.0128878951, 0.0567540489, ..."
2,309e9139,"[0.030787738, 0.0438391864, 0.0544221997, -0.0..."
3,7899f9e2,"[0.0632467642, 0.0684221983, 0.0797635466, 0.0..."
4,fde9c9f3,"[-0.0516408496, 0.00150877552, 0.0108443135, 0..."
...,...,...
9996,c25d98dc,"[-0.061014995, -0.0113675073, 0.00387942628, 0..."
9997,1fab6d6a,"[-0.0379287191, -0.022427015, -0.032286942, 0...."
9998,c2ae6461,"[-0.0503764041, -0.040498063, 0.0233088415, 0...."
9999,acaeab0e,"[-0.0025407786, -0.0498635843, -0.0346738808, ..."


In [89]:
p['embedding'] = p.apply(
    lambda row: update_dict.get(row['id'], row['embedding']),
    axis=1
)

In [91]:
p.to_csv("./data/profile_embedding.csv", index=False)